# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
!pip install selenium

In [2]:
! pip install chromedriver_autoinstaller

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
from selenium import webdriver
import chromedriver_autoinstaller
import time

In [5]:
# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
# chromedriver_autoinstaller.install()

In [6]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [7]:
# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [8]:
def get_response(request_url):
    headers =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    return requests.get(request_url, headers = headers)

In [9]:
def get_elements_by_attributes(source_data, tag, attributes):
    scrapped_data = BeautifulSoup(source_data, "html.parser")
    return scrapped_data.find_all(tag, attributes)

In [10]:
def get_elements_by_tag(source_data, tag):
    scrapped_data = BeautifulSoup(source_data, "html.parser")
    return scrapped_data.find_all(tag)

In [11]:
# Your code here
# A) Amazon Product - https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/
# Using webscrapping the data is being fetched and stored
# URL - https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_getr_d_paging_btm_prev_1?pageNumber=1

product_url = "https://www.amazon.com/Apple-MacBook-13-inch-256GB-Storage/product-reviews/B08N5M7S6K/ref=cm_cr_arp_d_paging_btm_{}?ie=UTF8&amp;pageNumber={}"

nextPage, current_page = True, 2
reviews, prev_cnt = [], 0
print("Collecting reviews")
while(nextPage and len(reviews) < 1000) :
    driver.get(product_url.format(current_page, current_page))
    response = driver.page_source
    page_content = BeautifulSoup(response, "html.parser")
    reviews.extend([i.text for i in page_content.find_all("span", {"class" : "a-size-base review-text review-text-content"})])
    print("-", end = "")
    current_page += 1
print(f"Total {len(reviews)} were collected.")
product_reviews_df = pd.DataFrame({"review" : reviews})
product_reviews_df.to_csv("/content/drive/My Drive/product_reviews.csv")

----------------------------------------------------------------------------------------------------Total 1000 were collected.


In [12]:
movie_review_url = "https://www.imdb.com/title/tt13751694/reviews?ref_=tt_urv"
reviews = []
while(len(reviews) < 1000):
    response_data = get_response(movie_review_url).text
    elements = get_elements_by_attributes(response_data, "div", {"class" : "text show-more__control"})
    pagination_key = get_elements_by_attributes(response_data, 'div',{'class' : 'load-more-data'})[0]["data-key"]
    reviews.extend([{"s.no" : len(reviews) + 1, "review" : i.text} for i, j in zip(elements, range(1, len(elements) + 1))])
    print(f"Collected - {len(reviews)}")
    movie_review_url = "https://www.imdb.com/title/tt13751694/reviews/_ajax?&paginationKey="+pagination_key
movie_reviews_df = pd.json_normalize(reviews)
movie_reviews_df.to_csv("/content/drive/My Drive/movie_reviews.csv")

Collected - 25
Collected - 50
Collected - 75
Collected - 100
Collected - 124
Collected - 149
Collected - 174
Collected - 199
Collected - 224
Collected - 249
Collected - 274
Collected - 299
Collected - 324
Collected - 349
Collected - 374
Collected - 399
Collected - 424
Collected - 449
Collected - 474
Collected - 499
Collected - 524
Collected - 549
Collected - 574
Collected - 599
Collected - 624
Collected - 649
Collected - 674
Collected - 699
Collected - 724
Collected - 749
Collected - 774
Collected - 799
Collected - 824
Collected - 849
Collected - 874
Collected - 899
Collected - 924
Collected - 949
Collected - 974
Collected - 999
Collected - 1024


In [13]:
g2_url = "https://www.g2.com/search?order=popular&page={}&query=software"
product, page = 0, 2
while(product < 1000):
    driver.get(g2_url.format(page))
    response = driver.page_source
    print(response)
    products = get_elements_by_attributes(response, "a", {"data-evt-name" : "engagement_product_click"})
    print(products)
    break

<html class="no-js" lang="en-US"><!--<![endif]--><head>
<title>Attention Required! | Cloudflare</title>
<meta charset="UTF-8">
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta http-equiv="X-UA-Compatible" content="IE=Edge">
<meta name="robots" content="noindex, nofollow">
<meta name="viewport" content="width=device-width,initial-scale=1">
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css">
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>


<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {
      var cookieEl = document.getElementById('cookie-alert');
      cookieEl.style.display = 'block';
    })
  }
</script>
<!--<![endif]-->


</head>
<body>
  <div id="cf-wrapper">
    <div class="cf-alert cf-alert-error cf-cookie-error" id="cookie-alert" data-tran

In [14]:
import json

semantic_scholar_url = 'https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract'
abstracts = []
while(len(abstracts) < 10000):
  print(semantic_scholar_url)
  response = json.loads(get_response(semantic_scholar_url).text)
  abstracts.extend(list(filter(lambda x : x is not None, [i["abstract"] for i in response.get("data", [])])))
  print(f"Collected Abstracts - {len(abstracts)}")
  semantic_scholar_url = f'https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract&token={response["token"]}'
abstracts_df = pd.DataFrame({"abstract" : abstracts})
abstracts_df.to_csv("/content/drive/My Drive/abstracts.csv")

https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract
Collected Abstracts - 770
https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract&token=PCOKWVSKJJGM4TWNJNI3EUSQJIWVNUSRKBFEYK2JFUBHFI4VBQGFFEZMKJGM2LCTJRJYZDINBVJEZTAN2LGA2TITJUWJHTATRWGAYU4NBRJE2DKTSRRGWAPMCIJ7G
Collected Abstracts - 1544
https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract&token=PCOKWVSKJJGM4TWNJNI3EUSQJIWVNUSRKBFEYK2JFUBHFI4VBQGFHDCNKMG5GTCMCPGREE2NJVJQ2DAMZUWVHTMNZRGC2TKNJQWYZTCN2OJMYLCNSRRGWAPGDEJX6
Collected Abstracts - 2307
https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="artificial intelligence"&fields=abstract&token=PCOKWVSKJJGM4TWNJNI3EUSQJIWVNUSRKBFEYK2JFUBHFI4VBSGIBEQWS3DESBTJS3E2M2MG4ZTOM2JJEZUQNBUWVELOMJUJFGLAMSMJFEUWNCKKWG2QB7OACT4Q
Collected Abstracts - 3063
https://api.semanticscholar.org/graph/v1/paper/search/bulk?query="arti

In [21]:
ddr_url = "https://ddr.densho.org/narrators/{}/"
infos, pages = [], 1
while(len(infos) < 904):
    response = get_response(ddr_url.format(pages))
    if(response.status_code != 200):
        pages += 1
        continue
    text = get_elements_by_tag(response.text, "p")[0].text
    if(len(text) != 0):
      infos.append(text)
    pages += 1
    infos = list(set(infos))
print(f"Collected - {len(infos)} Infos")
narrators_df = pd.DataFrame({"info" : infos})
narrators_df.to_csv("/content/drive/My Drive/narrators_info.csv")

Collected - 904 Infos


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [22]:
# Write code for each of the sub parts with proper comments.

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

class Processor:

    ps = PorterStemmer()
    ltzr = WordNetLemmatizer()
    stopwords = set(stopwords.words('english'))

    @classmethod
    def clean_text(cls, data):

        # Remove special characters
        data = re.sub(r'[^\w\s]', '', data)

        # Remove Digits
        data = re.sub(r'\d+', '', data)

        # Tokenization
        tokens = nltk.word_tokenize(data.lower())

        # Stopwords Removal
        tokens = [token for token in tokens if token not in cls.stopwords]

        # Stemming
        tokens = [cls.ps.stem(token) for token in tokens]

        # Lemmitization
        tokens = [cls.ltzr.lemmatize(token) for token in tokens]

        # Joining the tokens together and returning it.
        return ' '.join(tokens)

    @classmethod
    def process_data(cls, file_name):
        df = pd.read_csv(file_name)
        print(f"Read file - {file_name}")
        columns = df.columns
        new_col = f"Cleaned_{columns[-1]}"
        df[new_col] = df[columns[-1]].apply(cls.clean_text)
        file_name = file_name.split("/")
        file_name[-1] = "Cleaned_" + file_name[-1]
        del df[columns[0]]
        df.to_csv(f"{'/'.join(file_name)}")
        print(f"Generated file - {'/'.join(file_name)}")

file_names = ["/content/drive/My Drive/product_reviews.csv", "/content/drive/My Drive/movie_reviews.csv", "/content/drive/My Drive/narrators_info.csv", "/content/drive/My Drive/abstracts.csv"]
for file_name in file_names:
    Processor.process_data(file_name)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Read file - /content/drive/My Drive/product_reviews.csv
Generated file - /content/drive/My Drive/Cleaned_product_reviews.csv
Read file - /content/drive/My Drive/movie_reviews.csv
Generated file - /content/drive/My Drive/Cleaned_movie_reviews.csv
Read file - /content/drive/My Drive/narrators_info.csv
Generated file - /content/drive/My Drive/Cleaned_narrators_info.csv
Read file - /content/drive/My Drive/abstracts.csv
Generated file - /content/drive/My Drive/Cleaned_abstracts.csv


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [23]:
! pip install stanfordnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.4 MB/s eta 0:00:00


In [38]:
import spacy
from spacy import displacy
from nltk import Tree
import nltk
from spacy import displacy
from collections import Counter

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

nlp = spacy.load("en_core_web_sm")

def parts_of_speech_count(text):

    words = nltk.word_tokenize(text)

    pos_tags = nltk.pos_tag(words)

    pos_counts = Counter(tag[0] for _, tag in pos_tags if tag[0] in ['N', 'V', 'J', 'R'])

    return pos_counts

def named_entities_count(text):

    words = nltk.word_tokenize(text)

    pos_tags = nltk.pos_tag(words)

    named_entities = nltk.ne_chunk(pos_tags)

    entity_counts = Counter()
    for chunk in named_entities:
        if isinstance(chunk, nltk.tree.Tree):
            entity_type = chunk.label()
            if entity_type in ['PERSON', 'ORGANIZATION', 'LOCATION', 'PRODUCT', 'DATE']:
                entity_text = ' '.join([token for token, _ in chunk.leaves()])
                entity_counts[entity_type] += 1

    return entity_counts

def display_dependency_tree(sentence):
  doc = nlp(sentence)
  displacy.render(doc, style="dep", jupyter=True)

# Apply the preprocessing function to the processed column
file_names = ["/content/drive/My Drive/Cleaned_product_reviews.csv", "/content/drive/My Drive/Cleaned_movie_reviews.csv", "/content/drive/My Drive/Cleaned_narrators_info.csv"]
for file_name in file_names:
  print(f"Reading file - {file_name}")
  df = pd.read_csv(file_name)
  columns = df.columns
  first_value = df[columns[-1]][1]
  df['parts_of_speech'] = df[columns[-1]].apply(parts_of_speech_count)
  df['named_entities'] = df[columns[-1]].apply(named_entities_count)
  display_dependency_tree(first_value)
  temp_name = file_name.split("/")[-1]
  df.to_csv(file_name.replace(temp_name, f"PS_NE_{temp_name}"))
  print(f"Generated file - {file_name}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Reading file - /content/drive/My Drive/Cleaned_product_reviews.csv


Generated file - /content/drive/My Drive/Cleaned_product_reviews.csv
Reading file - /content/drive/My Drive/Cleaned_movie_reviews.csv


Generated file - /content/drive/My Drive/Cleaned_movie_reviews.csv
Reading file - /content/drive/My Drive/Cleaned_narrators_info.csv


Generated file - /content/drive/My Drive/Cleaned_narrators_info.csv


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [39]:
# Write your response below
# The data collection is a little bit challenging task. I tried checking for the open API's for data collection. But, i couldn't find any. Then i started exploring the different approaches to fetch the data.
# Capcha issue while trying to scrap data from the amazon.com and g2.com
# Not facing issue with amazon.com after using Selenium web driver
# semanticscholar public API is not working intermittently due to huge incoming traffic
# It's better to provide tasks on the publicly available sites or API's to scrap the Data. So that the scrapping can be done easily and the nlp processess will be done as per the statement.